In [ ]:
# Q1 part i

class Environment:
    def __init__(self, graph):
        self.graph = graph

    def get_percept(self, node):
        return node


class GoalBasedAgentDLS:
    def __init__(self, goal, depth_limit):
        self.goal = goal
        self.depth_limit = depth_limit

    def formulate_goal(self, percept):
        if percept == self.goal:
            return "Goal reached"
        return "Searching"

    def dls(self, graph, node, depth, path):
        if depth > self.depth_limit:
            return False

        path.append(node)

        if node == self.goal:
            return True

        for neighbor in graph.get(node, []):
            if neighbor not in path:
                if self.dls(graph, neighbor, depth + 1, path):
                    return True

        path.pop()
        return False

    def act(self, percept, graph):
        goal_status = self.formulate_goal(percept)

        if goal_status == "Goal reached":
            return f"Goal {self.goal} found!"

        path = []
        if self.dls(graph, percept, 0, path):
            return f"Goal found with DLS Agent. Path: {path}"
        else:
            return "Goal not found within depth limit."
        
    def run_agent(self, environment, start_node):
        print(f"Starting search from node: {start_node}")
        percept = environment.get_percept(start_node)
        result = self.act(percept, environment.graph)
        print(result)


# Main section
graph = {'A': ['B', 'C'], 'B': ['D', 'E'], 'C': ['F'], 'D': [], 'E': ['F', 'G'], 'F': [], 'G': ['H'] }

# Initialize environment and agent
env = Environment(graph)
agent = GoalBasedAgentDLS(goal='F', depth_limit=3)

# Run the agent from starting node 'A'
agent.run_agent(env, 'A')




Starting search from node: A
Goal found with DLS Agent. Path: ['A', 'B', 'E', 'F']


In [ ]:
# Q1 part ii

import heapq

class Environment:
    def __init__(self, graph):
        self.graph = graph

    def get_percept(self, node):
        return node


class UtilityBasedAgentUCS:
    def __init__(self, goal):
        self.goal = goal

    def ucs(self, graph, start_node):
        # Priority queue: (cumulative_cost, current_node, path_so_far)
        pq = []
        heapq.heappush(pq, (0, start_node, [start_node]))
        visited = set()

        while pq:
            cost, node, path = heapq.heappop(pq)

            if node == self.goal:
                return path, cost

            if node not in visited:
                visited.add(node)

                for neighbor, step_cost in graph.get(node, []):
                    if neighbor not in visited:
                        heapq.heappush(pq, (cost + step_cost, neighbor, path + [neighbor]))

        return None, None  # Goal not found

    def run_agent(self, environment, start_node):
        print(f"Starting UCS from node: {start_node} to goal: {self.goal}")
        percept = environment.get_percept(start_node)
        path, total_cost = self.ucs(environment.graph, percept)

        if path:
            print(f"Goal reached! Path: {path} | Total cost: {total_cost}")
        else:
            print("Goal not reachable from the start node.")


# Main Section
graph = {
    'A': [('B', 2), ('C', 5)],
    'B': [('D', 4), ('E', 1)],
    'C': [('F', 2)],
    'D': [],
    'E': [('F', 3)],
    'F': []
}

env = Environment(graph)
agent = UtilityBasedAgentUCS(goal='F')

# Run the agent from starting node 'A'
agent.run_agent(env, 'A')


Starting UCS from node: A to goal: F
Goal reached! Path: ['A', 'B', 'E', 'F'] | Total cost: 6


In [ ]:
import itertools

class Environment:
    def __init__(self, dist):
        self.dist = dist

    def get_cities(self):
        return list(self.dist.keys())

class TSPUtilityAgent:
    def __init__(self):
        self.best_route = None
        self.min_cost = float('inf')

    def utility(self, route, dist):
        cost = 0
        for i in range(len(route) - 1):
            cost += dist[route[i]][route[i+1]]
        return cost

    def run_agent(self, environment, start_city):
        cities = environment.get_cities()
        cities.remove(start_city)

        print("All possible routes and their costs:")
        for perm in itertools.permutations(cities):
            route = [start_city] + list(perm) + [start_city]
            cost = self.utility(route, environment.dist)
            print(f"Route: {route} | Cost: {cost}")
            if cost < self.min_cost:
                self.min_cost = cost
                self.best_route = route

        print("\nShortest route:")
        print(f"Route: {self.best_route} | Total distance: {self.min_cost}")


# Main Section
dist = {
    1: {2: 10, 3: 15, 4: 20},
    2: {1: 10, 3: 35, 4: 25},
    3: {1: 15, 2: 35, 4: 30},
    4: {1: 20, 2: 25, 3: 30}
}

env = Environment(dist)
agent = TSPUtilityAgent()
agent.run_agent(env, start_city=1)


All possible routes and their costs:
Route: [1, 2, 3, 4, 1] | Cost: 95
Route: [1, 2, 4, 3, 1] | Cost: 80
Route: [1, 3, 2, 4, 1] | Cost: 95
Route: [1, 3, 4, 2, 1] | Cost: 80
Route: [1, 4, 2, 3, 1] | Cost: 95
Route: [1, 4, 3, 2, 1] | Cost: 95

Shortest route:
Route: [1, 2, 4, 3, 1] | Total distance: 80


In [14]:
# Q3
class Environment:
    def __init__(self, graph):
        self.graph = graph

    def get_percept(self, node):
        return node


class IterativeDeepeningDFSAgent:
    def __init__(self, goal, max_depth=10):
        self.goal = goal
        self.max_depth = max_depth

    def dls(self, graph, node, depth, path):
        path.append(node)
        if node == self.goal:
            return True
        if depth <= 0:
            path.pop()
            return False
        for neighbor in graph.get(node, []):
            if neighbor not in path:
                if self.dls(graph, neighbor, depth - 1, path):
                    return True
        path.pop()
        return False

    def iddfs(self, graph, start_node):
        for depth in range(self.max_depth + 1):
            path = []
            found = self.dls(graph, start_node, depth, path)
            if found:
                return path, depth
        return None, None

    def run_agent(self, environment, start_node):
        percept = environment.get_percept(start_node)
        path, depth_reached = self.iddfs(environment.graph, percept)
        if path:
            print(f"Goal found! Path: {path} | Depth reached: {depth_reached}")
        else:
            print("Goal not reachable within max depth.")


graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F', 'G'],
    'F': [],
    'G': ['H', 'I']
}

tree = {
    '1': ['2', '3'],
    '2': ['4', '5'],
    '3': ['6', '7'],
    '4': [], '5': [], '6': [], '7': []
}

env_graph = Environment(graph)
agent_graph = IterativeDeepeningDFSAgent(goal='I', max_depth=5)
agent_graph.run_agent(env_graph, 'A')

env_tree = Environment(tree)
agent_tree = IterativeDeepeningDFSAgent(goal='7', max_depth=3)
agent_tree.run_agent(env_tree, '1')


Goal found! Path: ['A', 'B', 'E', 'G', 'I'] | Depth reached: 4
Goal found! Path: ['1', '3', '7'] | Depth reached: 2
